In [ ]:
#We won't be using this anymore. Sometimes Polygon API was giving us incomplete data so here was a a backup plan to pull the data from the API ourselves without using their function.
# but now the issu is solved
#

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from polygon import RESTClient
import datetime as dt
import os
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import os
import requests
import aiohttp
import asyncio
from datetime import datetime, timedelta

In [2]:
load_dotenv()
api_key = os.getenv("API_KEY")


In [3]:
tickers=['AAPL']

In [7]:
data_fetcher.get_urls('AAPL')

['https://api.polygon.io/v2/aggs/ticker/AAPL/range/5/minute/2003-01-01/2003-09-18?adjusted=true&sort=asc&limit=50000&apiKey=RpovRC4_0dsV0iW9wOYVpXYImBqXsY7v',
 'https://api.polygon.io/v2/aggs/ticker/AAPL/range/5/minute/2003-09-19/2004-06-05?adjusted=true&sort=asc&limit=50000&apiKey=RpovRC4_0dsV0iW9wOYVpXYImBqXsY7v',
 'https://api.polygon.io/v2/aggs/ticker/AAPL/range/5/minute/2004-06-06/2005-02-21?adjusted=true&sort=asc&limit=50000&apiKey=RpovRC4_0dsV0iW9wOYVpXYImBqXsY7v',
 'https://api.polygon.io/v2/aggs/ticker/AAPL/range/5/minute/2005-02-22/2005-11-09?adjusted=true&sort=asc&limit=50000&apiKey=RpovRC4_0dsV0iW9wOYVpXYImBqXsY7v',
 'https://api.polygon.io/v2/aggs/ticker/AAPL/range/5/minute/2005-11-10/2006-07-28?adjusted=true&sort=asc&limit=50000&apiKey=RpovRC4_0dsV0iW9wOYVpXYImBqXsY7v',
 'https://api.polygon.io/v2/aggs/ticker/AAPL/range/5/minute/2006-07-29/2007-04-15?adjusted=true&sort=asc&limit=50000&apiKey=RpovRC4_0dsV0iW9wOYVpXYImBqXsY7v',
 'https://api.polygon.io/v2/aggs/ticker/AAPL/r

In [4]:
class Fetching_Data:
    def __init__(self, tickers, trading_hours=16, start_date='2003-01-01', end_date='2024-02-01', timespan='minute', multiplier=5, limit=50000, api_key=api_key):
        self.tickers = tickers
        self.trading_hours = trading_hours
        self.start_date = datetime.strptime(start_date, '%Y-%m-%d')
        self.end_date = datetime.strptime(end_date, '%Y-%m-%d')
        self.timespan = timespan
        self.multiplier = multiplier
        self.limit = limit
        self.api_key = api_key

    def calculate_range(self): # THIS IS WORKING FINE
        if self.timespan == 'minute':
            return (self.limit // (self.trading_hours * 60)) * self.multiplier
        elif self.timespan == 'hour':
            return (self.limit // self.trading_hours) * self.multiplier
        elif self.timespan == 'day':
            return self.limit * self.multiplier

    def get_urls(self, ticker): # this is also working fine
        urls = []
        days_to_add = self.calculate_range()
        from_date = self.start_date
        while from_date < self.end_date:
            to_date = from_date + timedelta(days=days_to_add)
            if to_date > self.end_date:
                to_date = self.end_date
            url = f'https://api.polygon.io/v2/aggs/ticker/{ticker.strip()}/range/{self.multiplier}/{self.timespan}/{from_date.strftime("%Y-%m-%d")}/{to_date.strftime("%Y-%m-%d")}?adjusted=true&sort=asc&limit={self.limit}&apiKey={self.api_key}'
            urls.append(url)
            from_date = to_date + timedelta(days=1)
        return urls

    def fetch_data(self, url):
        response = requests.get(url)
        if response.status_code == 200:
            return response.json()
        else:
            return None


    def get_data(self):
        for ticker in self.tickers:
            urls = self.get_urls(ticker)
            all_data = pd.DataFrame()  # Initialize an empty DataFrame to store all data.
            for url in urls:
                data = self.fetch_data(url)
                if data and 'results' in data:  # Ensure there are results
                    df = pd.DataFrame(data['results'])
                    df['t'] = pd.to_datetime(df['t'], unit='ms').dt.tz_localize('UTC').dt.tz_convert('US/Eastern').dt.tz_localize(None)
                    df = df.rename(columns={'t': 'Date'}).set_index('Date')
                    df.drop(columns=['vw', 'o', 'h', 'l', 'v', 'n'], inplace=True)  # Adjust columns as needed
                    all_data = pd.concat([all_data, df])  # Append each DataFrame to the all_data DataFrame

            all_data.to_csv(f"{ticker.strip()}_data.csv", index=True)  # Save the combined DataFrame to CSV



                    


In [5]:
class DataPipeline:
    def __init__(self, tickers, start_date='2003-01-01', end_date='2024-02-01', timespan='minute', multiplier=5, limit=50000):
        self.tickers = tickers
        self.start_date = start_date
        self.end_date = end_date
        self.timespan = timespan
        self.multiplier = multiplier
        self.limit = limit
    
    def load_csv_data(self, ticker):
        
        try:
            data = pd.read_csv(f"{ticker.strip()}_data.csv")
            data.index = pd.to_datetime(data['t'], unit='ms').tz_localize('UTC').dt.tz_convert('US/Eastern').dt.tz_localize(None)
            data = data.rename(columns={'timestamp': 'Date'})
            filtered_data = data.between_time('09:30', '16:00').copy()
            # data = data.drop(columns=['o', 'vw', 'h', 'l', 'v', 't', 'n'])
            return data, filtered_data
        except FileNotFoundError:
            print(f"No CSV file found for {ticker}")
            return pd.DataFrame(), pd.DataFrame()

    def process_data(self, df):
        df['Day'] = pd.to_datetime(df.index).normalize()
        df['returns'] = df['close'].pct_change()
        df['log_ret'] = np.log(df['close']).diff()
        df['squared_ret'] = df['returns']**2
        df['squared_log_ret'] = df['log_ret']**2
        return df
    
    def calculate_realized_var(self, df):
        realized_var = df.groupby('Day').agg({'squared_ret': 'sum',
                                              'squared_log_ret': 'sum'})
        return realized_var.rename(columns={'squared_ret': 'daily_realised_var',
                                            'squared_log_ret': 'daily_realised_log_var'})

    def processing_pipeline(self):
        results = {}
        for ticker in self.tickers:
            data, filtered_data = self.load_csv_data(ticker)
            processed_data = self.process_data(data)
            processed_filtered_data = self.process_data(filtered_data)
            processed_data.to_csv(f'{ticker}_processed_data.csv')
            processed_filtered_data.to_csv(f'{ticker}_filtered_data_processed.csv')
            realized_var = self.calculate_realized_var(processed_data)
            realized_var_filtered = self.calculate_realized_var(processed_filtered_data).add_prefix('filtered_')
            final_var = realized_var.join(realized_var_filtered, how='outer')
            final_var.to_csv(f'{ticker}_realized_var.csv')
            results[ticker] = [realized_var, realized_var_filtered]
        return results


In [6]:
tickers=['AAPL']
data_fetcher = Fetching_Data(tickers)

# result = data_fetcher.get_urls()


In [ ]:
pipeline = DataPipeline(tickers)
results = pipeline.processing_pipeline()

In [21]:
try_url = data_fetcher.get_urls('AAPL')[0]